- Update config.yml
- Update params.yml
- Update entity
- Update the configuration manager in src config
- Update the components
- Update the pipeline
- Update main.py
- Update app.py

In [1]:
import os 
%pwd
os.chdir("../../../")
%pwd

'd:\\Project\\Text-Summarizer\\textSummarizer'

In [3]:
# Update entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTrasformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [4]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

# Update the configuration manager in src config
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifact_root])

    def get_data_transformation_config(self) -> DataTrasformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
    
        data_config = DataTrasformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_config


In [5]:
x = ConfigurationManager()
x = x.get_data_transformation_config()

[2025-04-07 14:13:17,596 - INFO - (common)]: YAML file config\config.yaml loaded successfully.
[2025-04-07 14:13:17,596 - INFO - (common)]: YAML file parameters.yaml loaded successfully.
[2025-04-07 14:13:17,596 - INFO - (common)]: Directory artifacts created or already exists.
[2025-04-07 14:13:17,596 - INFO - (common)]: Directory artifacts/data_transformation created or already exists.


In [6]:
# Update the components
import os
from textSummarizer.logging import logger
from transformers import  AutoTokenizer
from datasets import load_dataset, load_from_disk

c:\Users\manis\anaconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-07 14:13:23,035 - INFO - (config)]: PyTorch version 2.4.1+cu124 available.


In [7]:
class DataTransformation:
    def __init__(self, config: DataTrasformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def transform(self):
        logger.info("Transforming the dataset...")
        dataset = load_from_disk(self.config.data_path)
        dataset = dataset.map(self.convert_to_features, batched=True)
        dataset.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))
        logger.info("Dataset transformation completed.")
        

In [8]:
x = DataTransformation(x)
x = x.transform()

[2025-04-07 14:13:27,209 - INFO - (2742658353)]: Transforming the dataset...


Map:   0%|          | 0/819 [00:00<?, ? examples/s]c:\Users\manis\anaconda3\envs\textS\lib\site-packages\transformers\tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 91937.96 examples/s]

[2025-04-07 14:13:27,570 - INFO - (2742658353)]: Dataset transformation completed.


In [9]:
# Update the pipeline
try:
    config = ConfigurationManager()
    data_config = config.get_data_transformation_config()
    data_validation = DataTransformation(config=data_config)
    data_validation.transform()
except Exception as e:
    raise e

[2025-04-07 14:13:30,922 - INFO - (common)]: YAML file config\config.yaml loaded successfully.
[2025-04-07 14:13:30,924 - INFO - (common)]: YAML file parameters.yaml loaded successfully.
[2025-04-07 14:13:30,925 - INFO - (common)]: Directory artifacts created or already exists.
[2025-04-07 14:13:30,926 - INFO - (common)]: Directory artifacts/data_transformation created or already exists.
[2025-04-07 14:13:31,735 - INFO - (2742658353)]: Transforming the dataset...


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 135385.55 examples/s]

[2025-04-07 14:13:34,320 - INFO - (2742658353)]: Dataset transformation completed.
